In [2]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,extract_data_into_df,create_etl_watermark_table,insert_data_in_batches
from lookups import FileType
from prehook import return_csv_list,create_staging_tables
import prehook

db_session = create_connection()

In [3]:

stg_users_dataframe = pd.DataFrame({
     "age": [25, 30, 35],
     "gender": ['Male', 'Female', 'Male'],
     "fav_music_genre": ['Pop', 'Rock', 'Hip-Hop'],
     "music_time_slot": ['Morning', 'Evening', 'Afternoon'],
     "music_lis_frequency": ['Daily', 'Weekly', 'Monthly'],
     "music_satisfaction": ['High', 'Medium', 'Low']
 })

stg_songs_dataframe = pd.DataFrame({
     "track_id": [1, 2, 3],
     "track_name": ["Song1", "Song2", "Song3"],
     "track_artist": ["Artist1", "Artist2", "Artist3"],
     "track_popularity": [80, 90, 70],
     "track_album_id": [101, 102, 103],
     "track_album_name": ["Album1", "Album2", "Album3"],
     "track_album_release_date": ["2022-01-01", "2022-02-01", "2022-03-01"],
     "playlist_name": ["Playlist1", "Playlist2", "Playlist3"],
     "playlist_id": [201, 202, 203],
     "playlist_genre": ["Genre1", "Genre2", "Genre3"],
     "energy": [0.8, 0.7, 0.6],
     "key": [1, 2, 3],
     "loudness": [-5.0, -6.0, -7.0],
     "mode": [0, 1, 0],
     "speechiness": [0.1, 0.2, 0.3],
     "valence": [0.7, 0.6, 0.5],
     "tempo": [120, 130, 140],
     "duration_ms": [240000, 250000, 260000]
 })


In [4]:

stg_users_table_name = "stg_kaggle_spotify_users"
stg_users_schema_name = 'musicschema' 
stg_users_create_statement = return_create_statement_from_dataframe(
stg_users_dataframe, stg_users_schema_name, stg_users_table_name)

query = insert_statements_from_dataframe(stg_users_dataframe,stg_users_schema_name ,stg_users_table_name)

for statement in query:
    execute_query(db_session, statement)
    print(statement)
    
stg_songs_table_name = 'stg_kaggle_spotify_songs'
stg_songs_schema_name = 'musicschema'  
stg_songs_create_statement = return_create_statement_from_dataframe(
stg_songs_dataframe, stg_songs_schema_name, stg_songs_table_name)

execute_query(conn= db_session, query= stg_users_create_statement)

execute_query(conn= db_session, query= stg_songs_create_statement)

INSERT INTO musicschema.stg_kaggle_spotify_users (age, gender, fav_music_genre, music_time_slot, music_lis_frequency, music_satisfaction) VALUES (25, 'Male', 'Pop', 'Morning', 'Daily', 'High');
INSERT INTO musicschema.stg_kaggle_spotify_users (age, gender, fav_music_genre, music_time_slot, music_lis_frequency, music_satisfaction) VALUES (30, 'Female', 'Rock', 'Evening', 'Weekly', 'Medium');
INSERT INTO musicschema.stg_kaggle_spotify_users (age, gender, fav_music_genre, music_time_slot, music_lis_frequency, music_satisfaction) VALUES (35, 'Male', 'Hip-Hop', 'Afternoon', 'Monthly', 'Low');


In [5]:
from enum import Enum

class FileType(Enum):
    CSV = 1

datatype = FileType.CSV
datapath = "C:\\dataproject\\songs_dataset.csv"
df = extract_data_into_df(datatype,datapath)
df
#datapath2 = "C:\\dataproject\\users_dataset.csv"


In [6]:
# Create Dimension Tables
dim_users_data = stg_users_dataframe[["age", "gender", "fav_music_genre", "music_time_slot", "music_lis_frequency", "music_satisfaction"]]
dim_songs_data = stg_songs_dataframe[["track_id", "track_name", "track_artist", "playlist_genre"]]
dim_genre_data = stg_songs_dataframe[["playlist_genre"]]

dim_users_table_name = "Dim_users"
dim_songs_table_name = "Dim_songs"
dim_genre_table_name = "Dim_genre"

schema = 'musicschema'

# Execute create statements for dimension tables
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    dim_users_data, schema, dim_users_table_name))

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    dim_songs_data, schema, dim_songs_table_name))

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    dim_genre_data, schema, dim_genre_table_name))


In [7]:
# Create Fact Tables
fact_user_genre_data = stg_users_dataframe[["age", "gender", "fav_music_genre"]]
fact_user_genre_data["total_users"] = 1
fact_user_genre_data = fact_user_genre_data.groupby(["age", "gender", "fav_music_genre"]).sum().reset_index()

fact_song_genre_data = stg_songs_dataframe[["track_name", "track_artist", "playlist_genre"]]
fact_song_genre_data["total_songs"] = 1
fact_song_genre_data = fact_song_genre_data.groupby(["track_name", "track_artist", "playlist_genre"]).sum().reset_index()

fact_user_genre_table_name = "Fact_user_genre"
fact_song_genre_table_name = "Fact_song_genre"

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    fact_user_genre_data, schema, fact_user_genre_table_name))

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    fact_song_genre_data, schema, fact_song_genre_table_name))



C:\Users\Admin\AppData\Local\Temp\ipykernel_9448\3298915897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_user_genre_data["total_users"] = 1
C:\Users\Admin\AppData\Local\Temp\ipykernel_9448\3298915897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_song_genre_data["total_songs"] = 1


In [8]:
# 1. Total songs per genre
total_songs_per_genre_data = stg_songs_dataframe.groupby("playlist_genre").size().reset_index(name="total_songs")
total_songs_per_genre_table_name = "Fact_total_songs_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    total_songs_per_genre_data, schema, total_songs_per_genre_table_name))

# 2. Total users per genre
total_users_per_genre_data = stg_users_dataframe.groupby("fav_music_genre").size().reset_index(name="total_users")
total_users_per_genre_table_name = "Fact_total_users_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    total_users_per_genre_data, schema, total_users_per_genre_table_name))

# 3. Average listen time per genre (Assuming you have a "duration_ms" column in your SONGS table)
average_listen_time_per_genre_data = stg_songs_dataframe.groupby("playlist_genre")["duration_ms"].mean().reset_index(name="average_listen_time")
average_listen_time_per_genre_table_name = "Fact_average_listen_time_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    average_listen_time_per_genre_data, schema, average_listen_time_per_genre_table_name))


# 5. Age distribution per genre
age_distribution_per_genre_data = stg_users_dataframe.groupby(["fav_music_genre", "age"]).size().reset_index(name="count")
age_distribution_per_genre_table_name = "Fact_age_distribution_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    age_distribution_per_genre_data, schema, age_distribution_per_genre_table_name))


In [9]:
# Aggregate Tables
agg_genre_data = stg_songs_dataframe[["playlist_genre"]]
agg_genre_data["total_songs"] = 1
agg_genre_data = agg_genre_data.groupby("playlist_genre").sum().reset_index()

agg_dim_genre_data = stg_songs_dataframe[["playlist_genre"]]
agg_dim_genre_data = agg_dim_genre_data.drop_duplicates()

agg_total_songs_per_genre_table_name = "agg_total_songs_per_genre"
agg_dim_genre_table_name = "agg_dim_genre"


execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    agg_genre_data, schema, agg_total_songs_per_genre_table_name))



C:\Users\Admin\AppData\Local\Temp\ipykernel_9448\3565597171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_genre_data["total_songs"] = 1


In [10]:
# import prehook  

# csv_list = prehook.return_csv_list()
# staging_table_name = "songs"  # Replace with your desired staging table name

# # Test the create_staging_tables function
# prehook.create_staging_tables(csv_list, staging_table_name)


In [11]:
# watermark_table_sql_file ="C:\Data_project\SQL_Commands\V2_prehook_Create_ETL_watermark_table.sql"
# data = pd.read_csv("C:\dataproject\songs_dataset.csv")
# batch_size = 1000

# create_etl_watermark_table(db_session, watermark_table_sql_file)

# # Insert data in batches and record the ETL watermark timestamp
# insert_data_in_batches(data, stg_songs_schema_name, stg_songs_table_name, batch_size, 'etl_watermark')

# if db_session:
#     db_session.close()

Error creating ETL watermark table: [Errno 2] No such file or directory: 'C:\\Data_project\\SQL_Commands\\V2_prehook_Create_ETL_watermark_table.sql'
Error recording ETL watermark timestamp: descriptor 'strftime' for 'datetime.time' objects doesn't apply to a 'str' object
Error recording ETL watermark timestamp: descriptor 'strftime' for 'datetime.time' objects doesn't apply to a 'str' object
Error recording ETL watermark timestamp: descriptor 'strftime' for 'datetime.time' objects doesn't apply to a 'str' object
Error recording ETL watermark timestamp: descriptor 'strftime' for 'datetime.time' objects doesn't apply to a 'str' object
Error recording ETL watermark timestamp: descriptor 'strftime' for 'datetime.time' objects doesn't apply to a 'str' object
Error recording ETL watermark timestamp: descriptor 'strftime' for 'datetime.time' objects doesn't apply to a 'str' object
Error recording ETL watermark timestamp: descriptor 'strftime' for 'datetime.time' objects doesn't apply to a 'st

In [9]:
import pandas as pd
import re
#from pandas_handler import remove_special_characters
def remove_special_characters(dataframe, column):
    regex_pattern = '[@_!#$%^*()<>?/\\|}{~:-]'
    dataframe['new_' + column] = dataframe[column].apply(
        lambda x: re.sub(regex_pattern, '', str(x)) if pd.notna(x) else x)
    return dataframe

df = pd.read_csv('C:\dataproject\songs_dataset.csv')

column_to_clean = 'track_name'

df_cleaned = remove_special_characters(df, column_to_clean)

df_cleaned


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,energy,key,loudness,mode,speechiness,valence,tempo,duration_ms,new_track_name
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,6/14/2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,0.916,6,-2.634,1,0.0583,0.5180,122.036,194754,I Don't Care with Justin Bieber Loud Luxury R...
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),12/13/2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,0.815,11,-4.969,1,0.0373,0.6930,99.972,162600,Memories Dillon Francis Remix
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),7/5/2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,0.931,1,-3.432,0,0.0742,0.6130,124.008,176616,All the Time Don Diablo Remix
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,7/19/2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,0.930,7,-3.778,1,0.1020,0.2770,121.956,169093,Call You Mine Keanu Silva Remix
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),3/5/2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,0.833,1,-4.672,1,0.0359,0.7250,123.976,189052,Someone You Loved Future Humans Remix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23444,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),4/28/2014,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,0.922,2,-1.814,1,0.0936,0.2100,128.170,204375,City Of Lights Official Radio Edit
23445,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,3/8/2013,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,0.786,0,-4.462,1,0.0420,0.4000,128.041,353120,Closer Sultan & Ned Shepard Remix
23446,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),4/21/2014,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,0.821,6,-4.899,0,0.0481,0.4360,127.989,210112,Sweet Surrender Radio Edit
23447,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),1/1/2014,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,0.888,2,-3.361,1,0.1090,0.3080,128.008,367432,Only For You Maor Levi Remix
